#FINAL NOTEBOOK
## Names: 
Tuyet Nguyen

## Question: How does the distribution of first reviews compare to the distribution of all reviews for each user and for the top 10 categories and does geography play a role?

Yelp is a platform that allows users to write reviews about the businesses that they visited. Our purpose is to determine whether the users’ first reviews, which is the review that they write when they first create a Yelp account, will praise or pan the businesses.

## Main questions:

1. Do people join Yelp to praise or pan a business/categories?
2. Comparing the 1st review rating with the business/categories rating
3. What categories do people review for their 1st review?
4. Does the rating pattern vary geographically - different patterns different for U.S. and Canadian metro areas?

## How we will reach our solution using the data:

* What fields do we need from the Yelp data? 
* How do we find all the first reviews in the Yelp dataset? We can easily tell that which are the first reviews if the users only wrote one review. However, we would want to retrieve as much 1st reviews as possible. How can we do this?
* What regions and business categories we focus on? 
* What we need to do to compare the first review across these regions and categories?

## Why is this data relevant to our question?

#### User Data 
Needed to get an understanding of Yelp users' first reviews and the rating they give. This will help us see the distribution of users' first reviews and whether they tend to praise or pan a business. We may also want to see if there the date they joined Yelp (`yelping_since`) relates to when they write their review.

#### Review Data
Calculate average rating of a business's reviews to compare distribution of first reviews vs. overall reviews (whether they praise or pan the same business). The `date` column can also help us know how long it took them to write that review in comparison to the date they joined. It may help us know if there is a relationship between how long it took them to write their first review and what that rating is.

#### Business Data
Looking at categories and using `review_count` to get the top 10 most reviewed categories. We want to find the top-level categories using this dataset, but we also included the categories dataset in case we have to do it from there. We can also use the spatial data to answer the geography question and compare the distributions across metro areas.

#### Categories Data
To get the top-level categories. When joining with the business data, we can select the businesses that include those top-level categories.

###Loading the User data

***The user count: 1,968,703***

In [0]:
if ( spark.catalog._jcatalog.tableExists("users_table") ):
  df_users = spark.read.format('parquet').table("users_table")
  print("User count:", df_users.count() )
else:
  print("The Building Review and User Tables notebook has not been run, so the user data cannot be loaded.")

User count: 1968703

In [0]:
df_users.createOrReplaceTempView("users")

#Finding 1st reviews using the User dataframe
### ***Users who have written only one review:***
We want to look at users who have written only 1 review, and in order to do this we set a condition where the review count (number of written reviews) is equal to 1. This will show us the user's ID number and their account information. Knowing that these users only have 1 review, we also know that the rating average will be within the integer range values from 1 to 5. Looking at users with only 1 review ensures that we are looking at their absolute first and only review.

User_id, review_count, yelping_since, and stars are the columns that are selected to help us find out the information about 1st reviews.

In [0]:
df_1Review_Users = spark.sql("""
SELECT user_id, review_count, yelping_since, average_stars AS stars
FROM users
WHERE review_count = 1
ORDER BY yelping_since
""")
print("number of users with 1 Review: ", df_1Review_Users.count())
df_1Review_Users.show()

number of users with 1 Review: 348539
+--------------------+------------+-------------------+-----+
 user_id|review_count| yelping_since|stars|
+--------------------+------------+-------------------+-----+
qv0zg1b2C4nLzy73W...| 1|2004-10-12 18:44:11| 3.88|
uUqDkqrtILu8pY2hb...| 1|2004-11-22 13:49:27| 5.0|
umFlo4CN6CiOaMu7W...| 1|2004-12-22 23:12:53| 5.0|
j-W_cktNaR0Y7uRF1...| 1|2005-01-25 19:44:33| 4.0|
f4h3WIpcLdoksimQJ...| 1|2005-02-19 08:08:38| 5.0|
ibjGb6V_G1w3jsskX...| 1|2005-05-01 23:17:16| 2.0|
GgFpjQuK-Al_pc-YK...| 1|2005-05-21 14:56:10| 1.0|
UwLBtLWvpTCwnevs3...| 1|2005-06-09 17:56:00| 2.0|
r9WFkrrFpJVFWaRYe...| 1|2005-06-13 02:33:16| 5.0|
XoI23Je8uP3RQtVpD...| 1|2005-06-15 22:02:56| 3.59|
xRvgJTydEmqEEfd4z...| 1|2005-06-29 01:48:17| 5.0|
4oKFcK1uhw-9n49w2...| 1|2005-09-27 02:51:31| 5.0|
_Dwc1cDFCQx6B4q4_...| 1|2005-10-17 13:10:19| 5.0|
jIFSiar6ZT4CZPfLm...| 1|2005-11-08 17:14:25| 5.0|
rtGGNdFjhwB1tc7jL...| 1|2005-12-23 18:31:02| 3.0|
xLTZmUnu8sMuwWNyb...| 1|2005-12-27 19:54:32| 5.0|
NvFEDePhgsvXJ5lIJ...| 1|2006-01-17 22:07:07| 1.88|
Ph_PBXHwvCv6mFQ-2...| 1|2006-01-22 21:38:12| 5.0|
-BPrwFu1fi976ZVRG...| 1|2006-02-05 21:23:36| 4.5|
HGCO9yAaDtw_o4EeD...| 1|2006-02-08 03:35:51| 5.0|
+--------------------+------------+-------------------+-----+
only showing top 20 rows

#### Looking at the `stars` column, we can see that some users have only 1 review but their average rating is not in the integer range value of 1 to 5 stars. Yelp only allows reviews in the range of 1-5 stars so this does not make sense for users that have only written one review.

We want to identify how many of these users with only 1 review have an average rating with a decimal value. These decimal value reviews represent users who may have changed or deleted their initial review. 

By doing so, Yelp will know how many users changed their opinion and went back to delete or edit their 1st review. Yelp might want to analyze further on this segmentation dataset to gain insight into those users' behavior. Some questions might be useful such as looking into the businesses and those users' demographic to see if there is any pattern or correlation for them tend to change their opinion after leaving their first review. For example, is it certain regions, categories, or type of users that tend to go back to modify their 1st review? Similarly, do these modified reviews tend to be more positive or negative. For example, do users tend to delete/edit their first (negative) review when a business reaches out to them and resolve any problems that caused a user to leave a bad review.

Finally, Yelp may want to fix these overwrite records, they might want to save each review seperately instead of overwriting them in their back-end data or include an extra field in the dataset to identify reviews that are edited and the main reason users changed their mind about a business. By doing that, Yelp will be able to understand what motivate users to change their reviews instead of writting a new one.

For this step, the user_id, review_count, yelping_since, stars columns will be shown if the condition (of start does not equal to 1, 2, 3, 4, or 5) is met.

In [0]:
df_1Review_Users.createOrReplaceTempView("1Review_Users")

In [0]:
df_excluded_1Review_Users = spark.sql("""
SELECT user_id, review_count, yelping_since, stars
FROM 1Review_Users
WHERE stars != 1.0
AND stars != 2.0
AND stars != 3.0
AND stars != 4.0
AND stars != 5.0
ORDER BY yelping_since
""")
print("number of users excluded:", df_excluded_1Review_Users.count())
df_excluded_1Review_Users.show()
df_excluded_1Review_Users.unpersist()

number of users excluded: 4941
+--------------------+------------+-------------------+-----+
 user_id|review_count| yelping_since|stars|
+--------------------+------------+-------------------+-----+
qv0zg1b2C4nLzy73W...| 1|2004-10-12 18:44:11| 3.88|
XoI23Je8uP3RQtVpD...| 1|2005-06-15 22:02:56| 3.59|
NvFEDePhgsvXJ5lIJ...| 1|2006-01-17 22:07:07| 1.88|
-BPrwFu1fi976ZVRG...| 1|2006-02-05 21:23:36| 4.5|
rkjK3yZSbQfuM6Hid...| 1|2006-09-13 22:30:15| 2.86|
dpB_NZMlE7UEYsKyw...| 1|2006-09-26 17:50:25| 3.6|
jrclJr14Vm1OvEh0e...| 1|2006-11-29 18:24:59| 3.75|
rYFDXZFy8J_XPPBev...| 1|2007-03-09 01:45:08| 2.33|
ffyiRxo84emVW105u...| 1|2007-05-30 22:17:47| 2.33|
XFhrzTyZTlHPxq0xx...| 1|2007-06-09 12:46:29| 2.5|
LYxZZl4X3l5Aj-D30...| 1|2007-06-27 22:20:43| 4.5|
4VDaO5KhYmGxKPhIp...| 1|2007-07-08 23:49:48| 4.83|
qOEhq1JgdCL1q1ZHx...| 1|2007-10-12 15:54:12| 4.5|
l2fxw_R-xiDBVB9Gj...| 1|2007-11-12 00:40:27| 1.33|
ZFU0EjzpKUjgEce_4...| 1|2007-11-21 19:52:24| 4.67|
kWbGZF3kbTreifJgp...| 1|2008-01-18 07:32:18| 1.8|
9O-7e3TmzHQ4sBgDY...| 1|2008-02-19 16:37:00| 4.67|
KcWnZS5dLInuQD99O...| 1|2008-03-04 17:12:35| 1.5|
rWZuG7SqIf2cXzweS...| 1|2008-04-23 21:58:21| 4.5|
BeYpQbyHeQwqevXfx...| 1|2008-04-28 00:40:32| 4.2|
+--------------------+------------+-------------------+-----+
only showing top 20 rows

Out[21]: DataFrame[user_id: string, review_count: bigint, yelping_since: string, stars: double]

####Excluding the users' 1st reviews that have decimal values for their start ratings. 
Based on the result, about 1.4% (4,941 out of 348,539) of users who have written only 1 review have an average rating which are decimal values. We will exclude those users and only identify users who wrote 1 review and whose rating is a whole integer value within the allowed range of 1 to 5.

#### Based on the return values, there are 343,598 users who wrote 1 review only within the rating range 1 to 5. We might wonder whether they created an account just to give their opinions immediately or if there is any gap between the time they created a Yelp account vs the time they posted their 1st review. 
By answering this question, Yelp will be able to understand what motivates a User to create an account. Do individuals want to share their opinion or just simply want to create an account on Yelp for other purposes, such as viewing other users' reviews or discovering businesses near by.

In [0]:
df_selected_1Review_Users = spark.sql("""
SELECT user_id, review_count, yelping_since, stars
FROM 1Review_Users
WHERE stars = 1.0
Or stars = 2.0
Or stars = 3.0
Or stars = 4.0
Or stars = 5.0
""")
print("number of users selected:", df_selected_1Review_Users.count())
df_selected_1Review_Users.show()

number of users selected: 343598
+--------------------+------------+-------------------+-----+
 user_id|review_count| yelping_since|stars|
+--------------------+------------+-------------------+-----+
uUqDkqrtILu8pY2hb...| 1|2004-11-22 13:49:27| 5.0|
umFlo4CN6CiOaMu7W...| 1|2004-12-22 23:12:53| 5.0|
j-W_cktNaR0Y7uRF1...| 1|2005-01-25 19:44:33| 4.0|
f4h3WIpcLdoksimQJ...| 1|2005-02-19 08:08:38| 5.0|
ibjGb6V_G1w3jsskX...| 1|2005-05-01 23:17:16| 2.0|
GgFpjQuK-Al_pc-YK...| 1|2005-05-21 14:56:10| 1.0|
UwLBtLWvpTCwnevs3...| 1|2005-06-09 17:56:00| 2.0|
r9WFkrrFpJVFWaRYe...| 1|2005-06-13 02:33:16| 5.0|
xRvgJTydEmqEEfd4z...| 1|2005-06-29 01:48:17| 5.0|
4oKFcK1uhw-9n49w2...| 1|2005-09-27 02:51:31| 5.0|
_Dwc1cDFCQx6B4q4_...| 1|2005-10-17 13:10:19| 5.0|
jIFSiar6ZT4CZPfLm...| 1|2005-11-08 17:14:25| 5.0|
rtGGNdFjhwB1tc7jL...| 1|2005-12-23 18:31:02| 3.0|
xLTZmUnu8sMuwWNyb...| 1|2005-12-27 19:54:32| 5.0|
Ph_PBXHwvCv6mFQ-2...| 1|2006-01-22 21:38:12| 5.0|
HGCO9yAaDtw_o4EeD...| 1|2006-02-08 03:35:51| 5.0|
CRs-oqEjJhEdFnTTS...| 1|2006-02-08 21:43:28| 1.0|
ohXb5o6M2wWvUGSlO...| 1|2006-02-10 16:38:16| 5.0|
2OFaCw3qg0oga_FvJ...| 1|2006-02-18 00:32:02| 5.0|
qvAp_NOAUfxXsFFAq...| 1|2006-02-19 05:18:31| 3.0|
+--------------------+------------+-------------------+-----+
only showing top 20 rows

In [0]:
df_selected_1Review_Users.createOrReplaceTempView("1R_Users")

#Loading the Review data

***The review count: 8,021,122***

The next step after having the review count is to select columns from the review dataframe that can further help us answer our questions. Business_id, user_id, date, and stars are the four selected columns.

In [0]:
if ( spark.catalog._jcatalog.tableExists("reviews_without_text_table") ):
  df_reviews = spark.read.format('parquet').table("reviews_without_text_table")
  print("Review count:", df_reviews.count() )
else:
  print("The Building Review and User Tables notebook has not been run, so the review data cannot be loaded.")

Review count: 8021122

In [0]:
df_selected_review = df_reviews.select(df_reviews.business_id, df_reviews.user_id, df_reviews.date, df_reviews.stars)
df_selected_review.show()

+--------------------+--------------------+-------------------+-----+
 business_id| user_id| date|stars|
+--------------------+--------------------+-------------------+-----+
3chd-CjwQz7Pap_MQ...|c4Y3EX5aXYixkyVD6...|2019-09-04 19:16:55| 4.0|
XV45a3fz1d7SRoash...|fHTAUYCd5HuboQp6H...|2016-03-01 21:46:00| 3.0|
BxcLVgA6BndKDJol0...|ecXDhY91U-ORKk_U7...|2019-11-18 07:13:06| 3.0|
nSPh4_BKX3Fch0TOa...|ZKpETYqLymDFL76hw...|2019-08-15 16:30:53| 1.0|
KrcmoxVBX9o3OxFog...|zcp389V72h0DKl69X...|2019-10-24 20:37:02| 3.0|
ewn93D_Q6bnVosJtw...|xbksPRSmtsmDNiR1V...|2019-11-19 15:33:33| 2.0|
fKjeOlJTQ1W0cSDRY...|dG-J0IG122Nwqp0Co...|2019-10-21 00:15:38| 1.0|
b7xistZfcbSO6_0Mg...|O4ADr-eEYkP1OKQUx...|2017-12-22 07:26:39| 1.0|
6pM2YxQH8dQYs07_C...|LeSXKUJeFvX2aSvfP...|2019-04-15 14:20:28| 5.0|
iVjeR9z1xVt2ZpxVU...|_fX--5YqPXMB4b7q2...|2019-11-16 22:29:28| 5.0|
t0edCFMUYmFSN_eXT...|HhfNSpdHGSGiuyi1a...|2019-08-13 03:02:45| 4.0|
TumOl-iIDkZFRM1k9...|YKiRYAlL7SEV5P29p...|2019-03-15 20:26:47| 5.0|
ndjiWXoH1pPMwEBhR...|Zt_z6uOk0fPXjsFvC...|2019-11-19 14:47:04| 5.0|
z8lagvcw1Y4Pz4dVN...|y0HCy6GpDFnsQXhCG...|2019-11-24 18:17:55| 5.0|
0OZckGHdpbhMMy8ex...|SB7Ow6Y52kNcHdecd...|2018-04-06 03:23:10| 5.0|
d10IxZPirVJlOSpdR...|0SrL97xFh-mVMOxUF...|2017-03-10 03:47:23| 1.0|
J3EW9_IQQqkp4nN3q...|yZ3Zm1fBfqlBWhFik...|2015-04-06 00:50:59| 4.0|
E9HkUYYO_-5vKjvBw...|hRzmJCPmV1_ggM5jS...|2019-10-26 04:00:14| 5.0|
lNBhwf_3c0QBqBlsY...|HS3y3GltOH_pJ37-E...|2019-11-27 14:34:34| 4.0|
Kp4i7on7rxFRkWceK...|R-qaBztGNZoxEbKet...|2015-04-27 07:48:39| 5.0|
+--------------------+--------------------+-------------------+-----+
only showing top 20 rows

In [0]:
df_selected_review.createOrReplaceTempView("reviews")

##Comparison between the time Users joined Yelp VS. the time at which they wrote their 1st review


 #### Calculate the average rating for each business

This step is essential as we need the average rating for each business to compute the difference between user's rating vs. business's average.

In [0]:
df_bus_avg_stars = spark.sql("""
SELECT DISTINCT(business_id), ROUND(AVG(stars), 2) AS averageRating
FROM reviews
GROUP BY business_id
""")
print("number of businesses:", df_bus_avg_stars.count() )
df_bus_avg_stars.show()

number of businesses: 209393
+--------------------+-------------+
 business_id|averageRating|
+--------------------+-------------+
iLTwwBRj8qMRaecv2...| 4.37|
x6qH9HXhzuKM03jcZ...| 2.97|
ZXQdi91sr3SgWYzFS...| 2.91|
Jnqg1x-45knt1_R1L...| 3.78|
clXf1Kkl5ynDMBhKS...| 4.81|
1UwaMUnVKeWcV14qv...| 3.56|
Npsws-Pn0S7vtCKdx...| 4.45|
SjgeuBlgKER9yegpo...| 3.97|
y9dvqGszHniNqRrfR...| 4.77|
i51tBEdGn9hJMw0io...| 1.75|
llCxryWr8j1S39tus...| 4.44|
AA4lxVlnFnmQL5g0j...| 4.8|
WYCDOWMIS348c8GkF...| 1.5|
M7dW8WCWN7KlcdA2N...| 2.23|
W0ZNXoa93cjnKphZ1...| 4.49|
fMfJ0LUG5iCoFJIgH...| 5.0|
RsREl3LkFGiwbt3kG...| 2.86|
ytrjNSTjtkBopBLtH...| 3.15|
TdCyaV9ae0ykq94eM...| 3.57|
UhEhLSJrPRoiYWdHf...| 4.6|
+--------------------+-------------+
only showing top 20 rows

In [0]:
df_bus_avg_stars.createOrReplaceTempView("busAvgStars")

#### Identifying the dates when users' first reviews were written

Selecting required columns from each data set and combining them into a new table allows us to look at all the important data at once. This will also make calculating the data from different dataframes easier. In our case, we will be able to see the user_id, review_count, stars, and date columns of the first review dataset as well as the business_id and the date/time when that User joined Yelp, taken from the review dataset, in a new dataframe called "df_join_1R".

***We will then check if we missed any data in the User dataframe regarding users who only wrote one review***

In [0]:
df_join_1R = spark.sql("""
SELECT R.business_id, U.user_id, U.review_count, U.stars, R.date, U.yelping_since
FROM 1R_Users as U INNER JOIN reviews as R
ON U.user_id = R.user_id
ORDER BY date
""")
df_join_1R.show()

+--------------------+--------------------+------------+-----+-------------------+-------------------+
 business_id| user_id|review_count|stars| date| yelping_since|
+--------------------+--------------------+------------+-----+-------------------+-------------------+
v3AS5LGeV2Si4nOHZ...|rtGGNdFjhwB1tc7jL...| 1| 3.0|2005-12-23 18:34:26|2005-12-23 18:31:02|
AfoBKJuGBJvQhc3By...|xLTZmUnu8sMuwWNyb...| 1| 5.0|2005-12-27 20:02:00|2005-12-27 19:54:32|
7U7uLS9YjjhBM1Soy...|F-sO9nAoCoJ039bk7...| 1| 1.0|2006-03-27 00:02:01|2006-03-26 23:56:01|
xWVcdfpqgImFaD8kk...|khlgBM31omc6UtmxX...| 1| 1.0|2006-04-25 19:32:20|2006-04-25 19:24:49|
OqhMk3pIe-q-_ixiC...|ZIbfOHuh7UbhvwNcd...| 1| 4.0|2006-04-28 15:13:10|2006-04-28 15:03:23|
QEJgZqn_jYMvmkHaY...|VtaxyiPHGFFU-Mrju...| 1| 5.0|2006-09-01 18:16:22|2006-09-01 18:04:53|
KVsv8wRGnLX8QWoNZ...|ThdbXzqAtrePyYvkw...| 1| 1.0|2006-09-04 19:27:28|2006-09-04 19:02:59|
-FLnsWAa4AGEW4NgE...|Ezf8zkqNe2lVM6Q15...| 1| 5.0|2006-10-02 21:16:23|2006-10-02 21:08:15|
RvwZqjdkZ_pER0moP...|Mr10-iFwOwr_Ivvsx...| 1| 4.0|2007-02-14 05:03:02|2007-02-13 21:29:29|
tRYeAFJOwnDbSMWyX...|dsEu1uJzc144wgo2u...| 1| 5.0|2007-03-02 05:25:53|2007-03-02 05:21:31|
suJ-eo5Gkvcg0iXVQ...|wkLjOHddFo6BqOjLm...| 1| 5.0|2007-03-15 04:18:40|2007-03-15 04:12:37|
JmI9nslLD7KZqRr__...|3yybv8yACTtCofltE...| 1| 1.0|2007-03-26 16:59:49|2007-03-26 16:56:10|
EZ9Uf-lSRtj1fvdMQ...|XzB12MWsgX1Yy5Er5...| 1| 2.0|2007-05-11 23:05:34|2007-05-11 22:59:06|
OEi7JB1YJ-FBwzN8x...|Q1gSoS57rXypxGphE...| 1| 1.0|2007-05-17 02:03:00|2007-05-17 01:10:22|
juu7S1nP5eirbeJ8-...|-Yqxgs-e7FR3VYulk...| 1| 5.0|2007-06-03 09:22:25|2007-06-02 19:03:03|
Q7m8G4579-XYKV3LM...|cg5yY_y8JetzlWq5R...| 1| 5.0|2007-06-15 23:20:31|2007-02-09 23:10:03|
-LzUYsR54hd8cCiYX...|veo4bJGSiOPNvBVKA...| 1| 4.0|2007-06-21 16:44:45|2007-06-21 15:48:03|
SWjKi3VJzq72d277N...|VWSK3RXr98t78sAVr...| 1| 5.0|2007-07-26 21:11:49|2007-06-25 21:38:13|
cJFhtt3ROCT6vr9I-...|kUF9Iw-aRvAkstA9W...| 1| 5.0|2007-08-06 19:56:07|2007-04-14 17:43:06|
5KheTjYPu1HcQzQFt...|BnQomByQnL2hmniEi...| 1| 5.0|2007-08-08 02:45:27|2007-07-09 03:57:18|
+--------------------+--------------------+------------+-----+-------------------+-------------------+
only showing top 20 rows

In [0]:
df_missing = df_selected_review.join(df_selected_1Review_Users,df_selected_1Review_Users.yelping_since != df_selected_review.date, "left_anti")
print("Total missing users who wrote 1 review: ", df_missing.count())
df_missing.unpersist()

Total missing users who wrote 1 review: 0
Out[30]: DataFrame[business_id: string, user_id: string, date: string, stars: double]

### Difference between the day a User created a Yelp account vs the day they wrote their 1 review.
After finding the difference, we identify the total number of users in the dataset, the users who have a value of zero in the difference of days, and those have a value of greater than or equal to 1 in the difference of days. This will give us an idea of what motivates people to create a Yelp account; to instantly share their experience with a business or to just create an account to use the platform.

##### ***The results show that  122,560 out of 349,194 Users wrote their review the same day they joined Yelp, and there are 226,634 users who wrote their 1st review after their account was activated for more than 1 days.***

In [0]:
from pyspark.sql.functions import datediff,col
df_diffDays = df_join_1R.withColumn("diff_in_days", datediff(col("date"),col("yelping_since"))).orderBy(col("diff_in_days").desc())
print("Total users: ", df_diffDays.count())
print("Total users who have zero different days :",df_diffDays.filter(df_diffDays.diff_in_days == 0).count())
print("Total users have more than 1 different day in the record:", df_diffDays.filter(df_diffDays.diff_in_days > 0).count())
df_diffDays.show()

Total users: 349194
Total users who have zero different days : 122560
Total users have more than 1 different day in the record: 226634
+--------------------+--------------------+------------+-----+-------------------+-------------------+------------+
 business_id| user_id|review_count|stars| date| yelping_since|diff_in_days|
+--------------------+--------------------+------------+-----+-------------------+-------------------+------------+
_-OBx0-ReIMrPOmaR...|umFlo4CN6CiOaMu7W...| 1| 5.0|2019-11-06 02:35:11|2004-12-22 23:12:53| 5432|
9Z-oB9iAuKinWkC2U...|r9WFkrrFpJVFWaRYe...| 1| 5.0|2019-08-02 17:53:17|2005-06-13 02:33:16| 5163|
IWN2heYitkg-D4Udq...|GgFpjQuK-Al_pc-YK...| 1| 1.0|2019-04-01 18:35:56|2005-05-21 14:56:10| 5063|
VBhPgceaKWOXT71d3...|UwLBtLWvpTCwnevs3...| 1| 2.0|2018-08-05 15:29:32|2005-06-09 17:56:00| 4805|
82YGtjc5KKikNiqBZ...|QYDaGG_FyhpbvY2H_...| 1| 5.0|2019-04-22 19:22:45|2006-04-08 20:38:20| 4762|
KiJx5Y05fShCncZep...|ohXb5o6M2wWvUGSlO...| 1| 5.0|2019-01-06 14:54:41|2006-02-10 16:38:16| 4713|
0wW9PasC8pw8SY7rl...|XLaCIHOks7C-e37F4...| 1| 5.0|2019-07-31 19:15:26|2006-10-06 16:08:19| 4681|
m-2IuWscr1JgfPf4q...|XWkgIoapHGm_2WEUS...| 1| 1.0|2019-02-28 17:27:23|2006-07-10 12:49:55| 4616|
bE7UMQj0Fk3raBI7P...|vq9azW9hX6puQcoH0...| 1| 5.0|2019-12-10 19:35:38|2007-09-19 02:12:06| 4465|
V58EMoVyTThEaTdOu...|2UOjYHGjy3kCWYPQI...| 1| 5.0|2019-05-16 11:50:51|2007-03-08 20:29:50| 4452|
gx2yPrOJSwF1ApJYd...|jX2Cf8a3ntdQGj7MA...| 1| 5.0|2019-10-19 00:18:08|2007-09-13 21:10:01| 4419|
Kp4i7on7rxFRkWceK...|dTKKgn60ijcHeSkMQ...| 1| 5.0|2018-03-31 19:10:22|2006-03-14 14:17:54| 4400|
Q_AgXWopU53QpRthF...|cbiruwL8YqHjO8-l4...| 1| 5.0|2019-07-12 22:30:44|2007-07-17 02:35:00| 4378|
2UrIdz-tp3q_7R5u6...|tlPMgtV5lsb1TJWju...| 1| 5.0|2019-09-14 18:17:07|2007-11-09 02:55:17| 4327|
X9jyQFRSZgMuTYvxs...|Gz6PIiADm7YIXlZn_...| 1| 5.0|2018-10-10 18:33:23|2006-12-17 02:47:17| 4315|
nIBwnhQC-dKxVU5yD...|HGCO9yAaDtw_o4EeD...| 1| 5.0|2017-11-20 15:55:19|2006-02-08 03:35:51| 4303|
X9jyQFRSZgMuTYvxs...|Gz6PIiADm7YIXlZn_...| 1| 5.0|2018-09-20 16:12:52|2006-12-17 02:47:17| 4295|
6H8xfhoZ2IGa3eNiY...|sZwrBv0IP2-5Jd6zH...| 1| 5.0|2019-10-28 23:57:43|2008-01-29 08:20:57| 4290|
XA2ZDogxfv-1DVam9...|U3vNd_0K8cxNuyGZA...| 1| 5.0|2018-12-21 17:38:06|2007-04-16 00:02:06| 4267|
kiizdW8dFgBlND4mJ...|2wtvAcjNI2r5VdwB7...| 1| 5.0|2018-08-23 15:52:57|2007-01-07 17:14:00| 4246|
+--------------------+--------------------+------------+-----+-------------------+-------------------+------------+
only showing top 20 rows

In [0]:
df_diffDays.createOrReplaceTempView("1R_joined_UR")

####Here we will gain a better understanding of the amount of users who have no difference in the days between the day they created an account and wrote their first review and those users who have another value for their calculated difference in days.

In [0]:
df_dGap_1R = spark.sql("""
SELECT count(user_id) as Total_Users, stars,
IF(diff_in_days = 0, "Zero different day", "More than 1 different days") as Day_Distribution
FROM 1R_joined_UR
GROUP BY day_distribution, stars
ORDER BY stars
""")
display(df_dGap_1R)
df_dGap_1R.unpersist()

Total_Users,stars,Day_Distribution
68272,1.0,More than 1 different days
54507,1.0,Zero different day
13490,2.0,More than 1 different days
8274,2.0,Zero different day
10651,3.0,More than 1 different days
4078,3.0,Zero different day
8345,4.0,Zero different day
16790,4.0,More than 1 different days
47356,5.0,Zero different day
117431,5.0,More than 1 different days


Out[33]: DataFrame[Total_Users: bigint, stars: double, Day_Distribution: string]

#### ***Based on the chart, users who created their account on the same day tend to rate a business with 1 star (negative) compared to users who gave their first review later on (these users tend to give a business 5 stars)***

#### Given previous results, there are more people praising a business in general. Although, those who have written a review the same day their account is activated tend to pan businesses, other users tend to give higher ratings if they write their first review after their inital join date. Furthermore, we have also found one common behavior from users is that they tend to give extreme opinions on their 1st review: either 1 star or 5 stars since those two take up to 82% of the 1 reviews written dataset.

These results are based off data looking at the ratings of users who only wrote 1 review. What about the rating of users who wrote multiple reviews? In order for us to identify their first review, we need to join the User and Review datasets and assume that they wrote their first review on the same day that they created Yelp account.

#### Identifying 1st review from Users who have written multiple reviews

Create a new dataframe with that contains user_id, review_count, yelping_since, and stars where the review_count does not equal to 1 (or where users wrote more than 1 review)

In [0]:
df_moreThan1R_Users = spark.sql("""
SELECT user_id, review_count, yelping_since, average_stars AS stars
FROM users
WHERE review_count != 1
ORDER BY yelping_since
""")
print("number of users with more than 1 Review: ", df_moreThan1R_Users.count())
df_moreThan1R_Users.show()

number of users with more than 1 Review: 1620164
+--------------------+------------+-------------------+-----+
 user_id|review_count| yelping_since|stars|
+--------------------+------------+-------------------+-----+
c6HT44PKCaXqzN_Bd...| 108|2004-10-12 08:40:43| 4.14|
nkN_do3fJ9xekchVC...| 1363|2004-10-12 08:46:43| 3.7|
wqoXYLWmpkEH0YvTm...| 398|2004-10-12 08:51:07| 3.61|
sE3ge33huDcNJGW3V...| 11|2004-10-12 09:16:01| 4.64|
9549vDBw7ab0cuc93...| 21|2004-10-12 10:00:59| 3.92|
23J4vG9_xxxdnmi8C...| 1667|2004-10-12 12:29:35| 4.37|
npCUdiDUBmf5qVYDb...| 173|2004-10-12 14:41:17| 3.51|
0sidDfoTIHle5vvHE...| 55|2004-10-12 15:47:20| 3.83|
HU1VyFArIf7Ek7ILq...| 60|2004-10-12 16:14:49| 3.92|
5iOHz6pHmXi9SoB5q...| 108|2004-10-12 17:42:24| 3.55|
f_z2PCV2FLyXR5LzQ...| 18|2004-10-12 18:15:20| 4.33|
Dk2IkchUjADbrC05s...| 20|2004-10-12 19:53:36| 4.3|
yYSBB5q7bY-qSVvmM...| 6|2004-10-12 20:18:03| 4.14|
5IX9FOesKrLRyz618...| 1018|2004-10-12 20:23:21| 3.98|
m_aiUBgq6i-Zht5Lf...| 50|2004-10-12 21:40:36| 3.84|
A0e-tQr8h6nZ7a9Vz...| 223|2004-10-13 01:56:24| 3.12|
sYJjPRRpCB0Mm9f0Z...| 54|2004-10-13 02:22:57| 3.41|
_hS30zE8iGn5Ooknp...| 44|2004-10-13 03:57:25| 3.96|
qaJo-4bYfXk3LYYf-...| 28|2004-10-13 04:24:06| 3.9|
Oi26QGnPAEWdTUtil...| 56|2004-10-13 17:18:19| 3.93|
+--------------------+------------+-------------------+-----+
only showing top 20 rows

In [0]:
df_moreThan1R_Users.createOrReplaceTempView("moreThan1R_Users")

#### Finding first reviews from users with multiple reviews

***We will then get the earliest review from multiple reviews of each user***

In [0]:
df_multipleR_Users_joined = spark.sql("""
SELECT U.user_id, U.review_count, U.yelping_since, R.business_id, R.date, R.stars
FROM moreThan1R_Users AS U INNER JOIN reviews AS R
ON U.user_id = R.user_id
""")
print("number of users with multiple reviews:", df_multipleR_Users_joined.count() )
df_multipleR_Users_joined.show()

number of users with multiple reviews: 7666028
+--------------------+------------+-------------------+--------------------+-------------------+-----+
 user_id|review_count| yelping_since| business_id| date|stars|
+--------------------+------------+-------------------+--------------------+-------------------+-----+
--5Dnq-kuMq5bqvlg...| 8|2011-01-12 20:42:07|YYD8tl3ODut_D_sqd...|2014-07-23 02:57:42| 5.0|
--DvZltAxZ1sjFn6m...| 2|2019-02-02 22:43:19|2t-a8qkgoRiiohThD...|2019-02-02 22:51:46| 5.0|
--FoFf79nPpKASxw0...| 2|2018-04-23 02:01:22|qNYLt8zizn--wEpT1...|2019-05-06 18:29:22| 5.0|
--HeZYR8uZadRXLAQ...| 3|2014-09-03 15:14:23|_ZfjpSEO5ntk-1hbn...|2016-04-02 18:03:34| 3.0|
--HeZYR8uZadRXLAQ...| 3|2014-09-03 15:14:23|ebTvBxSStI9Vf5Tpu...|2016-09-13 13:21:46| 3.0|
--I4wRDhmM2J2VLzK...| 25|2014-01-17 01:04:30|JmI9nslLD7KZqRr__...|2014-09-14 23:33:43| 2.0|
--Oi-hOzz5vCLn9l6...| 11|2015-01-13 14:14:58|KligxMWsVPp7Jr_YZ...|2015-04-17 16:32:28| 4.0|
--PJ1FuEoTEo-3Cxf...| 28|2013-11-27 19:40:32|RESDUcs7fIiihp38-...|2016-08-11 20:51:16| 3.0|
--PJ1FuEoTEo-3Cxf...| 28|2013-11-27 19:40:32|2weQS-RnoOBhb1KsH...|2017-01-08 21:22:47| 2.0|
--RYvmB6UYRyZQqXk...| 4|2011-01-19 22:58:51|ZMbY3xmTJ3EB2f1ZF...|2011-08-29 16:41:33| 4.0|
--RYvmB6UYRyZQqXk...| 4|2011-01-19 22:58:51|MsUI2BgJjptqa1YEA...|2013-11-02 15:24:29| 1.0|
--RYvmB6UYRyZQqXk...| 4|2011-01-19 22:58:51|i7WdEjPrZkzo7whKd...|2013-01-10 02:50:28| 1.0|
--XsqVlXNtt20d5zS...| 4|2014-06-14 01:01:16|wk6aHP-vxv9dhmmJV...|2014-07-01 04:12:22| 1.0|
--XsqVlXNtt20d5zS...| 4|2014-06-14 01:01:16|3enxXjE0vtVaZ-un4...|2014-06-14 01:20:16| 1.0|
--XsqVlXNtt20d5zS...| 4|2014-06-14 01:01:16|zGXO7B16FXxjc_P0r...|2014-07-01 04:08:19| 2.0|
--YhC3mgAU7iexygW...| 4|2012-09-13 17:10:06|Xcal78l5dziYYUOfL...|2016-03-15 23:16:11| 4.0|
--agAy0vRYwG6WqbI...| 4|2010-07-17 17:11:53|ATeOjZzE3VOI-RW7F...|2013-11-26 22:46:06| 2.0|
--agAy0vRYwG6WqbI...| 4|2010-07-17 17:11:53|chmsA4G55YHyEMsVp...|2014-12-25 23:30:10| 4.0|
--bp0mKBU1gk9FUTS...| 3|2016-04-07 01:55:38|as_fyujgsdwtV56n3...|2016-04-07 02:07:56| 4.0|
--bp0mKBU1gk9FUTS...| 3|2016-04-07 01:55:38|GdzUXAqSPvH77jgQb...|2016-04-07 01:59:55| 1.0|
+--------------------+------------+-------------------+--------------------+-------------------+-----+
only showing top 20 rows

In [0]:
df_multipleR_Users_joined.createOrReplaceTempView("morethan1R_joined")

In [0]:
df_multipleR_Users_1st = spark.sql("""
SELECT row_num, user_id, review_count, business_id, yelping_since, date, stars
FROM (SELECT ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) row_num, business_id, user_id, review_count, yelping_since, date, stars FROM moreThan1R_joined)
WHERE row_num = 1
ORDER BY date
""")
print("total number of users with multiple reviews who have a 1st review:", df_multipleR_Users_1st.count() )
df_multipleR_Users_1st.show(40)

total number of users with multiple reviews who have a 1st review: 1620164
+-------+--------------------+------------+--------------------+-------------------+-------------------+-----+
row_num| user_id|review_count| business_id| yelping_since| date|stars|
+-------+--------------------+------------+--------------------+-------------------+-------------------+-----+
 1|sE3ge33huDcNJGW3V...| 11|PD2MAlYYi9HCqPH7I...|2004-10-12 09:16:01|2004-10-12 10:13:32| 5.0|
 1|nkN_do3fJ9xekchVC...| 1363|na4Th5DrNauOv-c43...|2004-10-12 08:46:43|2004-10-19 02:46:40| 5.0|
 1|yYSBB5q7bY-qSVvmM...| 6|GCRvrxMSC1nzShyM4...|2004-10-12 20:18:03|2004-10-19 03:05:42| 5.0|
 1|c6HT44PKCaXqzN_Bd...| 108|u8C8pRvaHXg3PgDrs...|2004-10-12 08:40:43|2004-10-19 09:07:15| 5.0|
 1|OIB57zcBHdv6omHyt...| 26|2Q2uE1dwhqMFOE5CY...|2004-10-25 04:01:44|2004-10-25 04:05:46| 5.0|
 1|62GNFh5FySkA3MbrQ...| 302|ydUqgWsF3F27TbauO...|2004-12-19 20:44:21|2004-12-19 20:47:24| 4.0|
 1|-xoI1iImivj9SgltL...| 16|iY_N3YhWL2_CUnBy4...|2004-12-20 03:01:48|2004-12-20 03:08:59| 5.0|
 1|23J4vG9_xxxdnmi8C...| 1667|ikubvyZFO0kxhA56R...|2004-10-12 12:29:35|2004-12-24 20:51:20| 5.0|
 1|Ud72j_rglYmXrL_O8...| 116|na4Th5DrNauOv-c43...|2005-01-14 01:17:58|2005-01-24 22:02:58| 5.0|
 1|0GMMy-FD8ImVsF4M6...| 140|LJvGjabschy4zGXwn...|2005-01-14 00:28:30|2005-01-26 02:49:45| 4.0|
 1|OJWl2qdzFM_IdTs23...| 2|Rg4FHrDOD2DJeDY_k...|2004-10-18 02:10:30|2005-02-02 06:53:48| 4.0|
 1|poDpBaq72hIBj3qIj...| 5|TCWMgOiV0PxQkWE1S...|2005-01-08 05:24:18|2005-02-07 05:12:21| 5.0|
 1|HU1VyFArIf7Ek7ILq...| 60|na4Th5DrNauOv-c43...|2004-10-12 16:14:49|2005-03-02 06:47:51| 5.0|
 1|HLaSqQMDVvlcFPGJL...| 179|VMPSdoBgJuyS9t_x_...|2005-03-03 19:40:10|2005-03-03 19:49:39| 5.0|
 1|ElAQK5tbY7GkmoJHM...| 5|AfoBKJuGBJvQhc3By...|2005-03-04 00:57:46|2005-03-04 01:06:19| 4.0|
 1|7VjGynQ1oCHZlytlm...| 238|pSQFynH1VxkfSmehR...|2005-01-06 21:12:51|2005-03-08 22:19:43| 5.0|
 1|cT0xyr130ZUpEcj8L...| 36|QiVnISL6SmxmahnW_...|2005-03-10 01:23:45|2005-03-10 02:07:19| 1.0|
 1|Jqs3IIVQCPA3bU5Ee...| 11|ues2rFe_hbRC7s232...|2005-03-11 20:52:50|2005-03-11 21:24:28| 5.0|
 1|GXn4ZsasLKh0qZ5g3...| 629|ITieHQ8UwKq74FnqT...|2005-01-03 18:01:35|2005-03-14 18:50:04| 5.0|
 1|lZsusXJjzStoi3vQm...| 259|h9ELiqDfwNktCvLEu...|2005-02-23 16:13:50|2005-03-16 17:08:51| 4.0|
 1|lrMmvkfLbx2YZkiJw...| 201|0FrYsoVHheQGoXEQs...|2005-03-14 03:06:57|2005-03-20 20:49:21| 4.0|
 1|pK-aWPU1g8sI93jb3...| 130|_DPJ_PHJnXsuhYhsg...|2005-03-28 18:52:30|2005-03-29 21:15:22| 5.0|
 1|qxC3xdg8UUKdNK9ci...| 111|na4Th5DrNauOv-c43...|2005-03-30 21:28:11|2005-04-01 00:38:02| 5.0|
 1|xcZo4LD1AgwH-aeKD...| 53|PGUuVJEzM0H-ZMB6m...|2005-03-09 23:08:02|2005-04-01 23:33:12| 4.0|
 1|msBFTEQdSkj_Vc_Vj...| 59|pSQFynH1VxkfSmehR...|2005-04-05 04:43:43|2005-04-08 20:38:20| 5.0|
 1|Lm0fu69svp2VsZY-_...| 32|gpcBFpC2umQ9sZfCO...|2005-03-05 00:37:52|2005-04-13 01:13:35| 2.0|
 1|Bf87HcPERF9yiSjb2...| 1594|LsjYGLrWe6psx1y5m...|2005-01-11 08:06:00|2005-04-13 03:48:03| 5.0|
 1|M36eKrxmWHYJvzZs8...| 160|Cr4ipI-tr-R8Q0Nil...|2005-01-24 22:54:49|2005-04-13 20:22:40| 5.0|
 1|ZftKc54UnKJSbuzyE...| 20|9IZX6ctv4ea6mc9gz...|2005-04-10 22:27:41|2005-04-15 01:46:33| 4.0|
 1|kgHcYU5dZRgwZQFoV...| 148|ZpMoI5EECS6Rm7Kig...|2005-04-14 17:21:03|2005-04-15 23:08:57| 3.0|
 1|NkVYcoaThHjOfTTgq...| 626|Wxxvi3LZbHNIDwJ-Z...|2005-04-20 16:57:48|2005-04-22 20:07:39| 5.0|
 1|AXFMaN-tq73Atxeep...| 56|na4Th5DrNauOv-c43...|2005-04-20 13:40:33|2005-04-23 19:35:12| 5.0|
 1|Gw4UiyZjER7GuyGbw...| 541|ya-2Hi4o1Ojo2AUBR...|2005-04-04 05:41:06|2005-04-27 23:29:03| 4.0|
 1|G5LXEaxhQMF_BVjNH...| 1905|6xgcHeLad-VaoTIQe...|2004-12-07 16:42:55|2005-04-27 23:38:05| 5.0|
 1|nzsv-p1O8gCfP3Xij...| 4117|BjH8Xepc10i6OhCDQ...|2005-04-12 03:08:20|2005-04-28 04:42:38| 4.0|
 1|VL8apbjr8DO12StXw...| 36|_4Gpn_NIM94D9OFuc...|2005-02-25 23:57:25|2005-05-02 19:00:24| 5.0|
 1|aIk07jxkia9izLMKR...| 124|_4Gpn_NIM94D9OFuc...|2005-02-23 21:05:59|2005-05-03 00:10:43| 3.0|
 1|KALxScy9eKOsR6YlA...| 57|ODTI9W7fHMKNWZ3g8...|2005-04-24 05:33:28|2005-05-04 00:56:46| 5.0|


#####Calculating the gap between the day they created the account vs their earliest written review, similar to the way we calculated these values for Users who have written only one review

As the result, we have ***263,078 users out of 1,620,164*** who wrote their earliest review the same day they joined Yelp, while the majority: 1,356,949 of users decided to write/edit their 1st review after their account was activated more than 1 day.

##### We will then identify whether the earliest reviews are also the users' 1st reviews by setting the differences in days equal to 0

In [0]:
df_difDays = df_multipleR_Users_1st.withColumn("diff_in_days", datediff(col("date"),col("yelping_since"))).orderBy(col("diff_in_days").desc())
print("Total users: ", df_difDays.count())
print("Total users have zero different day :",df_difDays.filter(df_difDays.diff_in_days == 0).count())
print("Total users have more than 1 different day in the record:", df_difDays.filter(df_difDays.diff_in_days > 0).count())
df_difDays.show()

Total users: 1620164
Total users have zero different day : 263078
Total users have more than 1 different day in the record: 1356949
+-------+--------------------+------------+--------------------+-------------------+-------------------+-----+------------+
row_num| user_id|review_count| business_id| yelping_since| date|stars|diff_in_days|
+-------+--------------------+------------+--------------------+-------------------+-------------------+-----+------------+
 1|LxJkyNF4T2Hy8Q-Hb...| 34|gh0mBl_SBZ6kX9mYM...|2004-10-28 20:22:45|2019-09-14 18:29:56| 1.0| 5434|
 1|kKEfxLoXYjIQ9KaE2...| 64|po_e1TQBimPN6cptd...|2005-03-21 19:33:29|2019-11-16 03:14:08| 4.0| 5353|
 1|0L4FdDkoGX6WLFyT4...| 211|EnQWjif1fdbzQcuBE...|2004-11-12 15:44:06|2019-05-06 13:38:47| 4.0| 5288|
 1|3MxHcBgASasAJb07v...| 76|uAXxdx-7Cn_X585Jb...|2005-07-06 23:07:07|2019-09-22 02:27:26| 5.0| 5191|
 1|PvkugmmO4_BcvDi-T...| 16|pH_qWwURHklnPmT_A...|2005-07-19 14:52:36|2019-09-15 01:01:19| 1.0| 5171|
 1|wHgdionsbDPZhD91B...| 5|kOsLCcNDWii4Bg4ev...|2005-05-31 23:14:13|2019-07-25 20:20:43| 5.0| 5168|
 1|Uo_nb-wB8fxoM6OuP...| 26|U21YaTK4PVlfNHQSy...|2005-05-25 07:31:29|2019-06-13 02:38:34| 5.0| 5132|
 1|1oesrOIBYtaJUr2Ug...| 301|OJZlwt18PpLg5Wy5_...|2005-08-29 20:17:35|2019-09-13 20:45:38| 5.0| 5128|
 1|QKjzJglNMT_m4XK60...| 96|1xExbYcT2SnnjyPXV...|2005-10-12 22:46:21|2019-10-01 05:11:14| 5.0| 5102|
 1|HZJ-5trEFnOsKycMn...| 29|MaRMFXMveB0unl04x...|2005-08-25 15:11:14|2019-08-06 21:20:29| 5.0| 5094|
 1|7SmgP46zfYvUT-Nem...| 89|jqAdZf9PJo8lsRskc...|2005-04-20 16:27:17|2019-03-16 00:11:22| 5.0| 5078|
 1|sVa3XiJaLgmIbLqLZ...| 134|744U45t4T97Q-0wqr...|2005-07-08 18:40:47|2019-06-03 07:47:55| 4.0| 5078|
 1|JXHEdrmKU-N_OX6h4...| 107|8M-fofsvdyflfUSoo...|2006-01-19 18:39:58|2019-12-12 01:25:34| 5.0| 5075|
 1|02l3vTUkz6Cy2DoMZ...| 9|EvOHshSlR5t9NwhUu...|2005-02-12 21:25:25|2018-12-30 00:17:38| 4.0| 5069|
 1|b9l5dws5A_o3HaLbO...| 36|iCQpiavjjPzJ5_3gP...|2005-08-16 13:11:26|2019-06-27 23:46:38| 3.0| 5063|
 1|Z76gsHaX5Ld5SXOwu...| 5|wZukjLaf1V2dLRQap...|2005-03-23 17:56:37|2019-01-29 05:47:22| 5.0| 5060|
 1|NYoJyRoWCubDSNyKy...| 152|JXwyTKI7TlnleDhT3...|2004-10-18 20:13:34|2018-08-24 05:57:52| 4.0| 5058|
 1|7FbmN_2BJ3m0rf2Df...| 46|xVEtGucSRLk5pxxN0...|2005-01-15 01:18:13|2018-11-20 21:48:01| 1.0| 5057|
 1|LnJCXi53V13_TIfke...| 22|tgFQQ4AK42geNLEME...|2006-02-28 04:42:33|2019-11-18 18:56:32| 3.0| 5011|
 1|00hTIAj8PYj5vbaVt...| 64|ixAh9crILnJ9tM8Lh...|2004-10-18 18:26:57|2018-07-02 17:08:32| 5.0| 5005|
+-------+--------------------+------------+--------------------+-------------------+-------------------+-----+------------+
only showing top 20 rows

In [0]:
df_0_difDay = df_difDays.select(df_difDays.business_id, df_difDays.user_id, df_difDays.review_count, df_difDays.stars, df_difDays.date, df_difDays.yelping_since, df_difDays.diff_in_days).filter((df_difDays.diff_in_days) == '0')
print("total number of users who created account and wrote the 1st review the same day", df_0_difDay.count())
df_0_difDay.show()

total number of users who created account and wrote the 1st review the same day 263078
+--------------------+--------------------+------------+-----+-------------------+-------------------+------------+
 business_id| user_id|review_count|stars| date| yelping_since|diff_in_days|
+--------------------+--------------------+------------+-----+-------------------+-------------------+------------+
tMxzAzHSFJWjSu6CE...|--GwB-sktmoAOPBsb...| 2| 5.0|2014-08-27 19:34:46|2014-08-27 19:33:36| 0|
WLz3ajQ8FIEPb1LAC...|--JqEn95hN31sznM1...| 13| 5.0|2014-09-26 14:39:58|2014-09-26 14:39:23| 0|
kd1VNJdd92T2eQ4hq...|--h2-gMvGgfBOmvlq...| 7| 1.0|2015-11-20 05:45:35|2015-11-20 05:45:30| 0|
8edTRAKWTzd0B_sfb...|-0KSt8Kw_WGcuy4Ak...| 6| 5.0|2010-08-03 18:46:29|2010-08-03 01:43:09| 0|
3xUoXuEgJhHlSgBsS...|-0OE9Pn8vSK-WjJeR...| 8| 2.0|2015-04-18 05:33:19|2015-04-18 05:13:26| 0|
3Nm15wEuq5k9iitPa...|-0RvEViQvvPLqrXKw...| 20| 4.0|2010-05-11 15:58:31|2010-05-11 15:19:44| 0|
ErxACeMyF4hSQyGhp...|-0S_XaK3Q_Mesal2U...| 37| 4.0|2009-02-24 04:31:42|2009-02-24 01:03:48| 0|
pImcSPg5tEkuvqHF7...|-1Dv-hZQUGPpmty6l...| 7| 5.0|2015-02-03 02:35:05|2015-02-03 02:30:22| 0|
LMrIDb9U2FYHr3HXz...|-1GRDbsno7wX3e6cJ...| 4| 5.0|2014-01-17 05:02:01|2014-01-17 04:44:38| 0|
sjeL8c_L0keFUTlY9...|-1wbglcr6x1qrUbqP...| 67| 5.0|2015-02-18 08:04:18|2015-02-18 07:52:22| 0|
QUs_7nqxo0xYE-Kaz...|-2hnChFYM5IgCkxEK...| 4| 5.0|2017-11-11 21:47:43|2017-11-11 21:34:50| 0|
qdHFUvFhZjvNie3cZ...|-2sVouJy7ObZQAaLl...| 3| 5.0|2018-08-17 19:34:14|2018-08-17 19:30:20| 0|
X5uxdU9GHoUAqo2wm...|-3DZPriPNwasJpm7P...| 2| 5.0|2012-02-02 17:01:03|2012-02-02 16:59:37| 0|
wEc4JE2NYQr1RnGjG...|-3d4B3pTm9aDe5RYz...| 2| 5.0|2017-08-19 21:01:03|2017-08-19 20:57:01| 0|
YrAbF_a8KCGdEz-Qu...|-3ilEC2Zn95K_Pcwp...| 2| 5.0|2018-03-02 22:21:50|2018-03-02 22:21:48| 0|
Y-S2LhHefBdnX8VP5...|-3z-rkvetwEcDfiwm...| 6| 2.0|2015-04-27 23:18:58|2015-04-27 22:04:32| 0|
HHcuXarbh_kJQ95Yj...|-4O6JQE31vUsc9nyt...| 2| 1.0|2017-04-23 11:55:10|2017-04-23 11:55:04| 0|
umxN12ok_qOB4qru0...|-4cGISljPoTmkKnms...| 4| 1.0|2015-08-05 17:03:18|2015-08-05 17:01:37| 0|
qjnJFZtsY_nfRzoL3...|-5_r9ZYM0PWSDNfju...| 2| 1.0|2017-02-12 02:44:37|2017-02-12 02:44:34| 0|
bBw1Lprmxc2kY59a-...|-5qroYdsEuZ4hK-bc...| 2| 5.0|2013-08-05 12:18:45|2013-08-05 11:45:07| 0|
+--------------------+--------------------+------------+-----+-------------------+-------------------+------------+
only showing top 20 rows

In [0]:
df_0_difDay.createOrReplaceTempView("multi_1stR")

#### Identifying total first-review users

We joined the 1st_review dataset that users only wrote one review with the 1st_review dataset that users have multiple reviews to combine them.

In [0]:
df_finalJ_1stR = spark.sql("""
SELECT U.business_id, U.user_id, U.review_count, U.stars, U.yelping_since, U.diff_in_days
FROM 1R_joined_UR as U 
UNION
SELECT R.business_id, R.user_id, R.review_count, R.stars, R.yelping_since, R.diff_in_days
FROM multi_1stR as R
ORDER BY diff_in_days
""").cache()
print("total 1review users", df_finalJ_1stR.count())
df_finalJ_1stR.show()

total 1review users 612192
+--------------------+--------------------+------------+-----+-------------------+------------+
 business_id| user_id|review_count|stars| yelping_since|diff_in_days|
+--------------------+--------------------+------------+-----+-------------------+------------+
VR1cbxGi6yzkrouTs...|1k1AN_3NseVO6lfo-...| 1| 5.0|2014-11-22 01:55:06| 0|
y-1sMhUMubrHWdIW4...|ERvW8o-gQMRx8s2W-...| 1| 5.0|2015-08-30 02:13:17| 0|
1muT5zEqoC4qp9ms0...|YOZGbKmL4OojtlVrT...| 1| 3.0|2017-06-16 22:29:50| 0|
7VfktTKK-ixMGdS2K...|YulxmSEpF5eAvFXk5...| 1| 5.0|2016-11-21 15:31:44| 0|
59S8mQtPgVdcsCp2a...|_4cwPUX2--8eFf1av...| 1| 1.0|2019-05-01 04:09:18| 0|
ymFexh_XrHSaM1dUq...|_KCF31HB3zpeYEoS_...| 1| 5.0|2016-10-14 02:13:23| 0|
xjUy3O9sVz8qFxBS8...|kFah14mbDfhrs7h0P...| 1| 2.0|2019-12-12 21:33:41| 0|
H9mHjdGNJpYN7yzgW...|6sBP_zz24bW4IwQbA...| 1| 4.0|2011-11-10 23:49:29| 0|
oMkFZJK82hNMxFKRL...|Rxj37K8nNFe-bsrYu...| 1| 5.0|2018-08-02 02:36:35| 0|
_PcRyvbZmVQvlS5uO...|p-fs5hOOzdo3XWEbi...| 1| 5.0|2016-11-19 03:08:09| 0|
R2qcJ_b1qrOkQT6ca...|yZoGwvsB7Lt3iSZlc...| 1| 4.0|2010-08-15 15:49:37| 0|
Vhh9MJrrtCoa-bVaJ...|IF-Og1hPX1PNg69Xr...| 1| 5.0|2019-11-14 15:22:02| 0|
laUfHtX3lervYx_i7...|QQMEz3f8haIwT6qTJ...| 1| 5.0|2019-07-14 18:59:34| 0|
bFeRYpzZ81w4y-Vaz...|VxQb1-o0TSjiguCmI...| 1| 5.0|2015-12-14 00:07:42| 0|
UvfBeHic40rwnxFBv...|r0qbiIvnh2nWaAG-9...| 1| 1.0|2013-07-01 19:19:36| 0|
hkoqV51IV875Jdzsp...|1NaUjom7RSpUiN9Xe...| 1| 2.0|2017-02-23 23:45:54| 0|
Rs2nW-8nT9c3kwJzc...|wyNDYYnGMO9ss0_Oi...| 1| 1.0|2019-04-07 17:53:18| 0|
UIU7tug_Y-qVv_aLt...|E2DN_KPVcf3HMuVKo...| 1| 5.0|2014-08-08 02:06:15| 0|
ypJMOo1eNGreGMfP6...|Ph5WizDbB5k9_1gDT...| 1| 5.0|2018-03-15 22:50:27| 0|
NqYttaW7CjJjhCN3Q...|p5hj-smjEjGowhK7T...| 1| 1.0|2018-10-22 22:38:46| 0|
+--------------------+--------------------+------------+-----+-------------------+------------+
only showing top 20 rows

In [0]:
df_finalJ_1stR.createOrReplaceTempView("finalJ_1stR")

In [0]:
df_1stRatings_vs_busAvg = spark.sql("""
SELECT 1stRtg.business_id, user_id, review_count, stars, averageRating,
  ROUND((stars - averageRating),2) AS rating_diff, yelping_since, diff_in_days
FROM finalJ_1stR AS 1stRtg LEFT OUTER JOIN busAvgStars AS busRtg
ON 1stRtg.business_id = busRtg.business_id
""")
df_1stRatings_vs_busAvg.show()

+--------------------+--------------------+------------+-----+-------------+-----------+-------------------+------------+
 business_id| user_id|review_count|stars|averageRating|rating_diff| yelping_since|diff_in_days|
+--------------------+--------------------+------------+-----+-------------+-----------+-------------------+------------+
3lFUdYf2zfFxrm8LI...|fNd91kyQk34860U3m...| 1| 1.0| 1.71| -0.71|2016-01-31 03:22:27| 0|
b62-BDE8LuBWGgNgZ...|C-fKIgeQF2Hpq6Hen...| 1| 1.0| 2.79| -1.79|2017-05-15 21:21:46| 0|
1hBmklxIBhw9nu9Nj...|PXK6u_7VclSO3LR-w...| 1| 2.0| 3.27| -1.27|2018-07-08 15:00:20| 0|
VHsNB3pdGVcRgs6C3...|MSAmHwyQEDcgrrNxX...| 1| 5.0| 3.41| 1.59|2017-08-20 01:09:41| 0|
JLbgvGM4FXh9zNP4O...|E5wcwxSYbtWQxwA5C...| 1| 5.0| 4.12| 0.88|2017-01-28 13:14:48| 0|
OyhCNhJJuJjUt2avx...|icP6ujyUmLufZgHOi...| 1| 1.0| 2.9| -1.9|2012-10-07 05:30:12| 0|
x_vNQ5dCJBLHsl-Fp...|atfQuS7mwd-2PHAtV...| 1| 1.0| 4.0| -3.0|2019-09-05 02:35:07| 0|
JLbgvGM4FXh9zNP4O...|jSaQ7bsGit5hC41-A...| 5| 5.0| 4.12| 0.88|2012-07-27 17:48:37| 0|
oeRLD870Z76FD1OYW...|EsApTSK4fx0NBar4l...| 15| 5.0| 4.73| 0.27|2011-03-07 06:37:25| 0|
M_w6EsZT4IrfA4jxp...|Bq4SPIgbo_-xJ6my7...| 8| 4.0| 4.35| -0.35|2015-09-03 23:55:30| 0|
RtUvSWO_UZ8V3Wpj0...|o8OL8qP-jvWNt7SLD...| 59| 4.0| 4.13| -0.13|2010-06-17 13:44:34| 0|
35X1ZV9tSEqB__yJE...|Ywan0XE4l16P11IU0...| 8| 5.0| 3.03| 1.97|2012-10-02 16:59:51| 0|
GGxnlrfvWy7LFvjN5...|YmycJBVpaAqd3L6xK...| 44| 1.0| 4.09| -3.09|2010-09-12 15:51:55| 0|
RtUvSWO_UZ8V3Wpj0...|g1ljhk93Gq8sGrp8Q...| 2| 3.0| 4.13| -1.13|2017-11-08 18:54:16| 0|
oIEmXWLtoh5blz-iw...|qSAUMZukrjolhzgy6...| 1| 2.0| 3.59| -1.59|2015-09-03 20:29:31| 0|
mHb5PqEb2qETQEPQX...|Vy3G82nTCCqldrVpj...| 1| 1.0| 2.86| -1.86|2017-07-30 05:25:49| 0|
35X1ZV9tSEqB__yJE...|2-IO4Oq-T9xMXJOnN...| 1| 5.0| 3.03| 1.97|2013-06-27 01:07:50| 0|
_Z3uT_S66jD9Pr9jF...|vZiejhfjqbeCrbuCm...| 1| 5.0| 3.71| 1.29|2019-09-12 18:21:35| 0|
2UgRg5a6KmpbD_SZf...|g9y_beypdvY8XU1vs...| 19| 5.0| 4.35| 0.65|2018-03-27 22:25:36| 0|
yEtGKrzpREur6uYFQ...|31CGpEYLV856lLtja...| 3| 5.0| 4.44| 0.56|2014-10-05 00:30:37| 0|
+--------------------+--------------------+------------+-----+-------------+-----------+-------------------+------------+
only showing top 20 rows

In [0]:
df_1stRatings_vs_busAvg.createOrReplaceTempView("1stR_vs_busAvg")

#### We have found 1st reviews from ***612,192 users***, which includes Users who wrote only 1 review and those who wrote multiple reviews

##Next we compare the 1st review rating distribution with the total review rating.

#### Grouping together the number of reviews with each level of stars before we see it through a picturesque feel.

In the other words, for both of the 1st reviews and the other reviews (not 1st reviews), we will compute the total number of reviews based on the star (1 to 5) that users have rated.

In [0]:
df_total_bus_rating = spark.sql("""
SELECT count(user_id) as other_reviews, ROUND(averageRating, 0) as stars 
FROM 1stR_vs_busAvg
GROUP BY ROUND(averageRating, 0)
ORDER BY ROUND(averageRating, 0)
""")
df_total_bus_rating.show()

+-------------+-----+
other_reviews|stars|
+-------------+-----+
 16816| 1.0|
 67523| 2.0|
 172920| 3.0|
 285513| 4.0|
 69420| 5.0|
+-------------+-----+

In [0]:
df_1R_rating = spark.sql("""
SELECT count(user_id) 1st_review, stars as 1R_stars
FROM finalJ_1stR
Group By stars
ORDER BY stars
""")
df_1R_rating.show()

+----------+--------+
1st_review|1R_stars|
+----------+--------+
 209980| 1.0|
 43628| 2.0|
 26957| 3.0|
 55265| 4.0|
 276362| 5.0|
+----------+--------+

In [0]:
df_total_bus_rating.createOrReplaceTempView("total_bus_rating")
df_1R_rating.createOrReplaceTempView("1R_rating")

#### Comparing the rating distribution between 1st review with the total reviews rating

This shows a side-by-side distribution for 1st review ratings and other review ratings.

We can see that for 1st reviews, users are on both extreme ends of the ratings. They either give 1 statr or 5 stars. The other reviews are showing the general distribution of what it looks like over time when all reviews are combined. We can see that the other reviews are on the higher end of the rating scale 

*Other_reviews does not include the 1st reviews. So we can see how the 1st review ratings heavily impact the distribution of other reviews.

In [0]:
df_rating_compare = spark.sql("""
SELECT T.other_reviews,  R.1st_review, T.stars
FROM total_bus_rating as T INNER JOIN 1R_rating as R
On T.stars =  R.1R_stars
ORDER BY T.stars
""")
display(df_rating_compare)
df_rating_compare.persist()

other_reviews,1st_review,stars
16816,209980,1.0
67523,43628,2.0
172920,26957,3.0
285513,55265,4.0
69420,276362,5.0


Out[49]: DataFrame[other_reviews: bigint, 1st_review: bigint, stars: double]

#### Importing business file
In this step, we import the business file into notebook, assign it to a new dataframe, and name it "df_business1". Then, we will start filtering the data by making changes (include or exclude certain information).

In [0]:
df_business1 = spark.read.json("/yelp/business.bz2")
print("business count:", df_business1.count() )
df_business1.printSchema()

business count: 209393
root
-- address: string (nullable = true)
-- attributes: struct (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- BusinessParking: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: string (nullable = true)
 |-- HappyHour: string (nullable = true)
 |-- HasTV: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- NoiseLevel: string (nullable = true)
 |-- Open24Hours: string (nullable = true)
 |-- OutdoorSeating: string (nullable = true)
 |-- RestaurantsAttire: string (nullable = true)
 |-- RestaurantsCounterService: string (nullable = true)
 |-- RestaurantsDelivery: string (nullable = true)
 |-- RestaurantsGoodForGroups: string (nullable = true)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- RestaurantsReservations: string (nullable = true)
 |-- RestaurantsTableService: string (nullable = true)
 |-- RestaurantsTakeOut: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- WheelchairAccessible: string (nullable = true)
 |-- WiFi: string (nullable = true)
-- business_id: string (nullable = true)
-- categories: string (nullable = true)
-- city: string (nullable = true)
-- hours: struct (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Thursday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
-- is_open: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- name: string (nullable = true)
-- postal_code: string (nullable = true)
-- review_count: long (nullable = true)
-- stars: double (nullable = true)
-- state: string (nullable = true)

#### Selecting "state" and rename SC to NC 
Similar to the other imported and assigned dataframes above, columns that could help with the analysis need to be retrieved and assign again to a new dataframe.We called this new dataframe "df_business2". We also need to rename SC to NC because when we look at metro areas, SC and NC are combined to be referred to as Charlotte.

In [0]:
import pyspark.sql.functions as f
 
df_business2 = df_business1.select("business_id","name","categories", "stars", "review_count" , "longitude", "latitude",\
                                   f.when(f.col("state") == 'SC', 'NC').otherwise( f.col("state")).alias("state")  ).cache()
print ("business count:", df_business2.count() )
df_business2.show(5, truncate=False)

business count: 209393
+----------------------+------------------------+--------------------------------------------------------------------------+-----+------------+------------+----------+-----+
business_id |name |categories |stars|review_count|longitude |latitude |state|
+----------------------+------------------------+--------------------------------------------------------------------------+-----+------------+------------+----------+-----+
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping |3.5 |36 |-80.8526119 |35.4627242|NC |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |Health & Medical, Fitness & Instruction, Yoga, Active Life, Pilates |5.0 |4 |-111.8902637|33.5694041|AZ |
XNoUzKckATkOD1hP6vghZg|Felinus |Pets, Pet Services, Pet Groomers |5.0 |5 |-73.58007 |45.479984 |QC |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |Hardware Stores, Home Services, Building Supplies, Home & Garden, Shopping|2.5 |3 |-115.1277255|36.2197281|NV |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |Home Services, Plumbing, Electricians, Handyman, Contractors |4.5 |26 |-111.7266485|33.4280652|AZ |
+----------------------+------------------------+--------------------------------------------------------------------------+-----+------------+------------+----------+-----+
only showing top 5 rows

#### Flattening the categories

Another step that was taken to filter business data is to flatten the categories. One business can have more than one category. Therefore, the function below will help display each business's categories in a seperate line instead of having them all in the same row. For example, if business_id #1 has both "Active Life" and "Shopping" categories, then the result will have two rows for this business_id (with different names for category column but the same information for other columns). 

After these adjustments are completed, the data will then be assigned to a new dataframe, called "df_business3".

In [0]:
import pyspark.sql.functions as f
 
df_business3 = df_business2.select("business_id","name","state", "stars", "review_count","longitude", "latitude" ,\
                                   f.explode(f.split(df_business2.categories, "\s*,\s*")).alias("category") ).cache()
print("record count:", df_business3.count() )
df_business3.show(truncate=False)

record count: 872794
+----------------------+------------------------+-----+-----+------------+------------+----------+---------------------+
business_id |name |state|stars|review_count|longitude |latitude |category |
+----------------------+------------------------+-----+-----+------------+------------+----------+---------------------+
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |3.5 |36 |-80.8526119 |35.4627242|Active Life |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |3.5 |36 |-80.8526119 |35.4627242|Gun/Rifle Ranges |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |3.5 |36 |-80.8526119 |35.4627242|Guns & Ammo |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |3.5 |36 |-80.8526119 |35.4627242|Shopping |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |5.0 |4 |-111.8902637|33.5694041|Health & Medical |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |5.0 |4 |-111.8902637|33.5694041|Fitness & Instruction|
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |5.0 |4 |-111.8902637|33.5694041|Yoga |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |5.0 |4 |-111.8902637|33.5694041|Active Life |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |5.0 |4 |-111.8902637|33.5694041|Pilates |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |5.0 |5 |-73.58007 |45.479984 |Pets |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |5.0 |5 |-73.58007 |45.479984 |Pet Services |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |5.0 |5 |-73.58007 |45.479984 |Pet Groomers |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |2.5 |3 |-115.1277255|36.2197281|Hardware Stores |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |2.5 |3 |-115.1277255|36.2197281|Home Services |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |2.5 |3 |-115.1277255|36.2197281|Building Supplies |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |2.5 |3 |-115.1277255|36.2197281|Home & Garden |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |2.5 |3 |-115.1277255|36.2197281|Shopping |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |4.5 |26 |-111.7266485|33.4280652|Home Services |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |4.5 |26 |-111.7266485|33.4280652|Plumbing |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |4.5 |26 |-111.7266485|33.4280652|Electricians |
+----------------------+------------------------+-----+-----+------------+------------+----------+---------------------+
only showing top 20 rows

#### Importing categories file

In [0]:
df_categories = spark.read.option("multiline","true").json("/yelp/categories.json")
print( "number of categories:", df_categories.count() )
df_categories.show()
df_categories.printSchema()

number of categories: 1565
+------------------+--------------------+-----------------+---------------+-------------------+
 alias| country_blacklist|country_whitelist| parents| title|
+------------------+--------------------+-----------------+---------------+-------------------+
 3dprinting| null| null|[localservices]| 3D Printing|
 abruzzese| null| [IT]| [italian]| Abruzzese|
 absinthebars| null| [CZ]| [bars]| Absinthe Bars|
 acaibowls|[PL, MX, AR, CL, ...| null| [food]| Acai Bowls|
 accessories| null| null| [fashion]| Accessories|
 accountants| null| null| [professional]| Accountants|
 acnetreatment| null| null| [beautysvc]| Acne Treatment|
 active| null| null| []| Active Life|
 acupuncture| null| null| [health]| Acupuncture|
 addictionmedicine| [JP]| null| [physicians]| Addiction Medicine|
 adoptionservices| null| null|[localservices]| Adoption Services|
 adult| null| null| [shopping]| Adult|
 adultedu| null| null| [education]| Adult Education|
adultentertainment| null| null| [nightlife]|Adult Entertainment|
 advertising| null| null| [professional]| Advertising|
 aerialfitness| null| null| [fitness]| Aerial Fitness|
 aerialtours| null| null| [tours]| Aerial Tours|
 aestheticians|[BE, MX, FR, CH, ...| null| [medicalspa]| Aestheticians|
 afghani| [MX, TR]| null| [restaurants]| Afghan|
 african| [TR]| null| [restaurants]| African|
+------------------+--------------------+-----------------+---------------+-------------------+
only showing top 20 rows

root
-- alias: string (nullable = true)
-- country_blacklist: array (nullable = true)
 |-- element: string (containsNull = true)
-- country_whitelist: array (nullable = true)
 |-- element: string (containsNull = true)
-- parents: array (nullable = true)
 |-- element: string (containsNull = true)
-- title: string (nullable = true)

#### Identifying top categories
This step shows us all the major categories in the categories dataset.

In [0]:
df_topCategories = df_categories.filter(f.size(df_categories.parents) == 0).select("title").cache()
df_topCategories.show(30,truncate=False)

+----------------------------+
title |
+----------------------------+
Active Life |
Arts & Entertainment |
Automotive |
Beauty & Spas |
Bicycles |
Education |
Event Planning & Services |
Financial Services |
Food |
Health & Medical |
Home Services |
Hotels & Travel |
Local Flavor |
Local Services |
Mass Media |
Nightlife |
Pets |
Professional Services |
Public Services & Government|
Religious Organizations |
Restaurants |
Shopping |
+----------------------------+

In [0]:
df_business3.createOrReplaceTempView("business")
df_topCategories.createOrReplaceTempView("categories")

#### Identifying top categories for each business

In [0]:
df_busCategories = spark.sql("""
SELECT B.*
FROM business AS B INNER JOIN categories AS C
ON B.category = C.title
""")
print("record count:", df_busCategories.count() )
df_busCategories.show(truncate=False)
df_business3.unpersist()
df_topCategories.unpersist()

record count: 295769
+----------------------+------------------------------+-----+-----+------------+--------------+-------------+----------------+
business_id |name |state|stars|review_count|longitude |latitude |category |
+----------------------+------------------------------+-----+-----+------------+--------------+-------------+----------------+
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman |NC |3.5 |36 |-80.8526119 |35.4627242 |Active Life |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman |NC |3.5 |36 |-80.8526119 |35.4627242 |Shopping |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |5.0 |4 |-111.8902637 |33.5694041 |Health & Medical|
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |5.0 |4 |-111.8902637 |33.5694041 |Active Life |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |5.0 |5 |-73.58007 |45.479984 |Pets |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |2.5 |3 |-115.1277255 |36.2197281 |Home Services |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |2.5 |3 |-115.1277255 |36.2197281 |Shopping |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |4.5 |26 |-111.7266485 |33.4280652 |Home Services |
cKyLV5oWZJ2NudWgqs8VZw|Oasis Auto Center - Gilbert |AZ |4.5 |38 |-111.8271417 |33.3503993 |Automotive |
oiAlXZPIFm2nBCt0DHLu_Q|Green World Cleaners |NV |3.5 |81 |-115.241463 |36.0639767 |Local Services |
ScYkbYNkDgCneBrD9vqhCQ|Junction Tire & Auto Service |AZ |5.0 |18 |-111.6822257 |33.3938847 |Automotive |
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |IL |4.5 |5 |-88.2330726 |40.1104457 |Food |
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |IL |4.5 |5 |-88.2330726 |40.1104457 |Restaurants |
EosRKXIGeSWFYWwpkbhNnA|Xtreme Couture |ON |3.0 |16 |-79.5291079302|43.6245394916|Active Life |
MbZMmwo-eL0Jnm_Yb9KJrA|Chinook Landscaping and Design|AB |5.0 |3 |-114.0372072 |50.9459599 |Home Services |
7Dv4_HAxsxvadEsT5fxQBg|Dependable Brakes & Exhaust |PA |5.0 |5 |-80.0044502 |40.4066674 |Automotive |
M_guz7Dj7hX0evS672wIwA|Chocolate Shoppe Ice Cream |WI |3.5 |6 |-89.5101417974|43.1053100891|Food |
JjJs3o60uQCfctDjs45cmA|Convertabath |AZ |2.5 |10 |-111.9516598 |33.3038687 |Home Services |
JjJs3o60uQCfctDjs45cmA|Convertabath |AZ |2.5 |10 |-111.9516598 |33.3038687 |Local Services |
kOICO53wbOiOJcKuCgOQ3A|Tan Las Vegas |NV |3.5 |5 |-115.1798386 |36.263362 |Beauty & Spas |
+----------------------+------------------------------+-----+-----+------------+--------------+-------------+----------------+
only showing top 20 rows

Out[56]: DataFrame[title: string]

#### Counting the number of categories for each business

A business can be in multiple top-level categories, like in the above cell. The first business is in 2 categories: Active Life and Shopping.

In [0]:
df_busCount = df_busCategories.groupBy("business_id").count().toDF("business_id", "catCount")
print("record count:", df_busCount.count() )
df_busCount.show(truncate=False)

record count: 208869
+----------------------+--------+
business_id |catCount|
+----------------------+--------+
M_guz7Dj7hX0evS672wIwA|1 |
oqbNgOBAxsa49c2WQWHL-w|2 |
2BKKUjLSocdlgfsMC2RSGQ|1 |
DloBjCZHlz0lzsrQJq5PHQ|3 |
lj0MiK5_fyv9df2twnsI7g|1 |
WtjgLPHTCYtqiZ31KBRDHQ|1 |
x1fx7C_tc064cFATjl4fmQ|3 |
s5345m9QBKwZrliFQ9FcfA|2 |
RasF3XQRsFqfmY596ixKhA|1 |
ATeOjZzE3VOI-RW7FUFJpQ|2 |
VHsNB3pdGVcRgs6C3jt6Zg|1 |
AucmLoCNGDkdg8sn6nxv1Q|1 |
RMjCnixEY5i12CiqnskiVw|1 |
BQMVcJQV70gUQjwi7-MrSg|1 |
M9N2eaO8PkT7MHGLLA6V3Q|1 |
wwl11NPZg4KehKNLx28_nw|2 |
YYztMMdcudL4xuSF29yelQ|1 |
aZbbt_RPKV8vUY7GWiEEyw|1 |
KqRn57aegoaDmbHqbqRwKw|2 |
ei7SYxd07uETpASn1KyY_g|1 |
+----------------------+--------+
only showing top 20 rows

#### Identifying missing records

In [0]:
df_missing = df_business2.join(df_busCount,df_business2.business_id == df_busCount.business_id,"left_anti")
print("records:", df_missing.count() )
df_missing.show(truncate=False)
df_business2.unpersist()

records: 524
+----------------------+-----------------------------------------+----------+-----+------------+------------+----------+-----+
business_id |name |categories|stars|review_count|longitude |latitude |state|
+----------------------+-----------------------------------------+----------+-----+------------+------------+----------+-----+
2W1tLg8ybRUEKMPoAPHTsQ|Cox Media |null |1.0 |3 |-115.1628395|36.0672701|NV |
oCbk_T5eyc3KB0iT6T5q9w|Criminal Customz |null |1.0 |4 |-82.171509 |41.461689 |OH |
4YI_gJQuW-8d_o2xvflJxA|Cag Acceptance |null |1.5 |6 |-111.9258563|33.4778079|AZ |
reOlnE49Ug3BMSGn3IAweg|Csorba Tree Experts |null |5.0 |3 |-79.8378623 |40.3553877|PA |
KnAV3ZhOWWNhqnvqIkxYOg|Accurate Lien & Contractor Assistance |null |1.5 |3 |-111.9481344|33.480885 |AZ |
pG8D1gthGbMMOj9y1MxOeA|Cub Cadet |null |1.5 |3 |-81.8944931 |41.2605173|OH |
3stT022u8m1H37mcopGuUA|Flik & Company |null |1.5 |4 |-79.3389433 |43.6612901|ON |
HgJqVPLNI29YJIlYBmScQQ|Ice Man of Arizona Equipment Company |null |2.0 |5 |-112.0808409|33.4262525|AZ |
_lVdg7Tdw-Vkn0dn-gS2qw|Vascular Access Center of Southern Nevada|null |4.0 |4 |-115.1847011|36.1596001|NV |
Fjd69v08ozCbV2qEbWkLCQ|Grant and Weber |null |1.0 |3 |-115.1236952|36.1697096|NV |
oYFUHj8XFM36SdPCoQaXPQ|Austin's Automotive Inspection Service |null |3.5 |3 |-80.6722684 |35.2080763|NC |
8eHUD8OBe2nXaLKV68K0Rw|Fortune Properties |null |1.0 |3 |-115.2438892|36.1537173|NV |
_idu2VZT3e2xy1tGx9P4fA|Greenshield Canada |null |1.0 |3 |-79.3850164 |43.6485025|ON |
5qv5MOZh1UTmj18-5zSusQ|Trugreen Chemlawn |null |1.0 |3 |-80.8942756 |35.1861617|NC |
IBAvWgl1CX8doybLdA-74g|Peoples Utility |null |1.0 |3 |-115.2140187|36.1447994|NV |
pm36bCxGC_hPkLMhiObBDw|Arizona Pipeline Company |null |1.0 |3 |-112.126787 |33.441364 |AZ |
hEQxSi_Mo5xXtSFnK9trSw|City Tin Shop |null |3.0 |4 |-80.5757636 |35.4161926|NC |
9dAHyMaKyE1Rzl9U1AKaBQ|Davita #3682 |null |4.0 |4 |-115.1715467|36.146642 |NV |
rWsoiLQeFy-RCdYtQvH-JA|Budget Rent A Car System |null |1.0 |3 |-115.0478166|36.0989785|NV |
BiBTwivxbsVVlNVgj4lduA|M & D Limited |null |1.0 |3 |-81.5014198 |41.3606845|OH |
+----------------------+-----------------------------------------+----------+-----+------------+------------+----------+-----+
only showing top 20 rows

Out[58]: DataFrame[business_id: string, name: string, categories: string, stars: double, review_count: bigint, longitude: double, latitude: double, state: string]

#### Setting weights for the categories

We want to weigh the categories because a business may have multiple top-level categories. The category weights needs to be evenly distributed for each business so that there is equal contribution from each record being worked on.

In [0]:
df_busCategories.createOrReplaceTempView("busCat")
df_busCount.createOrReplaceTempView("busCount")
 
df_weighted = spark.sql("""
SELECT B.*, (1 / C.catCount) AS weight
FROM busCat AS B INNER JOIN busCount AS C
ON B.business_id = C.business_id
""").cache()
print("record count:", df_weighted.count() )
df_weighted.show(truncate=False)
df_busCategories.unpersist()

record count: 295769
+----------------------+----------------------------------------+-----+-----+------------+--------------+-------------+---------------------+------+
business_id |name |state|stars|review_count|longitude |latitude |category |weight|
+----------------------+----------------------------------------+-----+-----+------------+--------------+-------------+---------------------+------+
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |4.0 |1759 |-115.16919 |36.123183 |Restaurants |1.0 |
-6c_bJblLXUwoWfmYLtH_w|Neo-Paws International |ON |2.5 |12 |-79.4790443 |43.6267627 |Pets |1.0 |
-ElDqujEn1u64ynQr0bX7g|McFarling Law Group |NV |4.5 |16 |-115.2277697 |36.1298777 |Professional Services|1.0 |
-Gh9a15ijNii-8rnp4x3xA|Volvo of Mississauga |ON |3.5 |3 |-79.7059750579|43.6186366615|Automotive |1.0 |
-I06hkMFrX0KBqu61ik1YA|Andre Agassi College Preparatory Academy|NV |5.0 |3 |-115.1572419 |36.1952552 |Education |1.0 |
-Qb6U1MotJfpt6OU_uvqQA|Woof Wise Dog Training |OH |5.0 |3 |-81.6383131 |41.2406079 |Pets |1.0 |
-RRnldwSqCZT6OpuWKXolg|Grandiose |AZ |5.0 |3 |-112.0720814 |33.4556571 |Shopping |1.0 |
-Spbg8zeDHA49Ys_f8WEIw|The Elf Squad |NC |5.0 |3 |-80.8477081 |35.2215329 |Shopping |1.0 |
-TGwtTiieh_JLml5_VRMQA|Starbucks |ON |3.5 |8 |-79.441195 |43.884114 |Food |1.0 |
-VAsjhmAbKF3Pb_-8rh3xg|McDonald's |QC |2.0 |12 |-73.558392 |45.515767 |Restaurants |0.5 |
-VAsjhmAbKF3Pb_-8rh3xg|McDonald's |QC |2.0 |12 |-73.558392 |45.515767 |Food |0.5 |
-WGjtt88-6zBiIUiKhJjgg|The Nail Salon & Spa |ON |2.5 |3 |-78.947639 |43.8594156 |Beauty & Spas |1.0 |
-cxD1NimFldATDUsN-oa3A|Coasta Cantina |NV |2.0 |23 |-115.202016 |36.102737 |Restaurants |1.0 |
-m8v19CXrGEYr3PALjqVCg|Daystar Boutique |OH |4.0 |4 |-81.8093744 |41.4516622 |Shopping |1.0 |
-qL-wX-UuMaHDBA2X5G4kw|The Thumb |AZ |4.0 |124 |-111.8791228 |33.639246 |Automotive |1.0 |
-r8SvItXXG6_T3mP5GXRAw|Western Coffee Shop |AB |4.0 |12 |-114.0707963 |51.0340032 |Restaurants |0.5 |
-r8SvItXXG6_T3mP5GXRAw|Western Coffee Shop |AB |4.0 |12 |-114.0707963 |51.0340032 |Food |0.5 |
-tUFV7Zk3RCvPL8QIDG9Bg|Pure Healthy Cosmestic Spa |NV |5.0 |5 |-115.30355483 |36.2073361 |Health & Medical |0.5 |
-tUFV7Zk3RCvPL8QIDG9Bg|Pure Healthy Cosmestic Spa |NV |5.0 |5 |-115.30355483 |36.2073361 |Beauty & Spas |0.5 |
-wby-wyqaq1ZDTFDTdKxQA|Trendy Wall Designs |NV |3.0 |21 |-115.136389 |36.175 |Shopping |0.25 |
+----------------------+----------------------------------------+-----+-----+------------+--------------+-------------+---------------------+------+
only showing top 20 rows

Out[59]: DataFrame[business_id: string, name: string, state: string, stars: double, review_count: bigint, longitude: double, latitude: double, category: string]

In [0]:
df_weighted.createOrReplaceTempView("bus_cat")

### Which categories do users write reviews for?

One of the questions ask which categories users tend to write reviews for. We can see that the top 3 categories are: Restaurants, Shopping, and Food. This will let us know that these categories will impact the distribution greatly when we do comparisons by category.

In [0]:
most_reviewed_cat = spark.sql("""
SELECT category, COUNT(review_count) AS total_reviews
FROM bus_cat
GROUP BY category
ORDER BY total_reviews DESC
""")
most_reviewed_cat.show(22)
most_reviewed_cat.unpersist()

+--------------------+-------------+
 category|total_reviews|
+--------------------+-------------+
 Restaurants| 63944|
 Shopping| 34644|
 Food| 32991|
 Home Services| 22487|
 Beauty & Spas| 20520|
 Health & Medical| 19227|
 Local Services| 15783|
 Automotive| 14720|
 Nightlife| 14211|
Event Planning & ...| 11263|
 Active Life| 10225|
Professional Serv...| 6923|
Arts & Entertainment| 6802|
 Hotels & Travel| 6413|
 Pets| 4402|
 Education| 3763|
 Financial Services| 3529|
 Local Flavor| 1492|
Public Services &...| 1485|
Religious Organiz...| 616|
 Mass Media| 320|
 Bicycles| 9|
+--------------------+-------------+

Out[61]: DataFrame[category: string, total_reviews: bigint]

#### Creating the final table that we will bring into Tableau
###### ****Total number of records being brought into Tableau is: 946,950****

In [0]:
df_final_tableau = spark.sql("""
SELECT U.user_id, B.business_id, B.state, B.category,\
U.averageRating, U.stars, U.rating_diff, U.diff_in_days  
FROM bus_cat AS B INNER JOIN 1stR_vs_busAvg AS U
ON B.business_id = U.business_id
Order By diff_in_days
""").cache()
print("record count:", df_final_tableau.count() )
df_final_tableau.show(truncate=False)

record count: 946950
+----------------------+----------------------+-----+-----------+-------------+-----+-----------+------------+
user_id |business_id |state|category |averageRating|stars|rating_diff|diff_in_days|
+----------------------+----------------------+-----+-----------+-------------+-----+-----------+------------+
fycqueI1dhlxqExcqjyt4Q|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
5L6EfyJlJM_oa85hb3JCxw|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
EphmW35tOkHdhwObwqZFJQ|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
NKMIUN6l17TKeFN9V9aqYA|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
28JozPjjEzGqv4PBLGJL5A|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |1.0 |-3.12 |0 |
4MS9ikcvjds7BbygGgnk4w|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |1.0 |-3.12 |0 |
m2n2KDQIyqPzn9wU5GjALA|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
TbzOfYNd4QR17KhxjHzy4Q|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |4.0 |-0.12 |0 |
zBJGn10SfsO4Kaa8mkUpdA|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |2.0 |-2.12 |0 |
PTHCHcBhJbyNS3hMSQ2pYQ|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |3.0 |-1.12 |0 |
4pbbNuzxlZ9E52wsVhgGig|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |1.0 |-3.12 |0 |
7Sv3oaZk0BS3h0JjPovBdA|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |1.0 |-3.12 |0 |
vfwCVDpABZpKGT1OVDIuOg|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
fL640F9UUOIFM6uTcJaT_w|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
TZiE5oDHGbXrMrvPBaidPA|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |2.0 |-2.12 |0 |
TdwK9CyzBKRmEs_WJW0Ppw|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
zHvwZn2UWbed7CZjjrePFg|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
tMCnDhGGQ7UEWMqfgl1qng|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |1.0 |-3.12 |0 |
z5ziTpDwgJVW7jY-x1A9-Q|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |4.0 |-0.12 |0 |
Rz1Z9E2JjoI69dt1be20Vw|--9e1ONYQuAa-CB_Rrw7Tw|NV |Restaurants|4.12 |5.0 |0.88 |0 |
+----------------------+----------------------+-----+-----------+-------------+-----+-----------+------------+
only showing top 20 rows

In [0]:
df_final_tableau.write.mode("overwrite").saveAsTable("business_users")

# Analysis and Conclusion

In [0]:
%pip install Pillow

Python interpreter will be restarted.
Requirement already satisfied: Pillow in /local_disk0/.ephemeral_nfs/envs/pythonEnv-e40e9b99-b2e9-4046-94c2-daca29838fb6/lib/python3.8/site-packages (8.2.0)
WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-e40e9b99-b2e9-4046-94c2-daca29838fb6/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
from PIL import Image
TEMP_DIR = "/temp"

def getWidth(path):
  with Image.open(path) as img:
    width, height = img.size
    return(width)
    
def getDbfsPathName(path):
    # Get the fileinfo containing the path and name
  if path.startswith("/dbfs") != True:
    raise Exception("The path provided does not start with /dbfs")
  new_path = "dbfs:" + path[5:]
  # get the file info for the path
  file_list = dbutils.fs.ls(new_path)
  if len(file_list) != 1:
    raise Exception("The path provided is not a single file on dbfs")
  dbfs_path = file_list[0].path
  filename = file_list[0].name
  return(dbfs_path, filename)
  
def getTempPath(filename):
  # Create the temp directory if it does not exist
  temp_path = "file:" + TEMP_DIR
  dbutils.fs.mkdirs(temp_path)
  temp_list = dbutils.fs.ls(temp_path)
  # get a name to use for the copy
  temp_files = []
  for info in temp_list:
    temp_files.append(info.name)
  increment = 0
  new_name = filename
  while new_name in temp_files:
    increment+=1
    new_name = filename + "." + str(increment)
  access_path = TEMP_DIR + "/" + new_name # used for file opening
  return(access_path)
import base64
from PIL import Image
 
def showimage(path, width=0):
  image_string = ""
  img_tag = ""
  dbfs_path, filename = getDbfsPathName(path)
  access_path = getTempPath(filename)
  # copy the file
  copy_path = "file:" + access_path
  dbutils.fs.cp(dbfs_path,copy_path)
  with open(access_path, "rb") as image_file:
    image_string = base64.b64encode(image_file.read() ).decode('utf-8') 
    
  # Is the width setting a positive integer?  A width of 50 means 50%
  if width > 0 and width < 1:
    print("If the width parameter is specified, it must be 1 or more.  A width of 50 means 50%. The width entered was " + str(width) + ", so the original image width was used.")
    width = 0 #reset
    
  if width == 0:
    height = 0
    # Get the width and height of the image in pixels
    with Image.open(access_path) as img:
      width, height = img.size
      
    framewidth = width * 1.1
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  style="width:%ipx;height=%ipx;" /></div>''' % (framewidth,framewidth,image_string, width, height)
  else: # a width was specified
    originalWidth = getWidth(access_path)
    imagewidth = int( width / 100.0 * originalWidth)
    framewidth = int( imagewidth * 1.1 )
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  width="%ipx" height="auto"></div>''' % (framewidth,framewidth,image_string, imagewidth)
  # Clean up the file
  dbutils.fs.rm(copy_path)
  return(img_tag)

### <a href="https://public.tableau.com/views/YelpUsersFirstReviewsAnalysis/1stReviewsDistribution?:language=en&:display_count=y&publish=yes&:origin=viz_share_link" target= "_blank">Visualizations </a>

### Filtering Records in Tableau

Before creating the visualizations, we filtered out 2 fields:

* State: We selected the 10 states with the highest number of businesses. We also set aliases and changed the states to the name of their respective metro areas. 
* Category: We excluded 4 categories: Bicycles, Mass Media, Public Service, and Religious Organizations. We excluded these categories because they had very little data to provide valuable insight to our question.

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/1st_Reviews_Distribution.png", 50) )

### Visualization 1: Distribution of First Review Ratings

The column shelf contains the `stars` column, which has been renamed to 'Rating'. The rows column shows a count of `user_id`. 

Looking at the data it seems that our largest column, 5 stars, indicates that most users will join to strongly praise a business. There is a strong positive feeling with these users as they rate the business at the greater end of the scale. The second largest column is from users who joined to pan the business (give a 1 star rating). This is the lowest a user can rate a business which indicates users will write a review once they feel a strong negative emotion.

Our data shows us that users join to write a review when they feel strongly about a business. These feelings are mostly positive but as seen in the graph above can also be very negative. This is why the 5 star and 1 star columns are our greatest columns.

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/Rating_Difference_by_Category_.png", 50) )

### Visualization 2: Rating Difference by Category

The column shelf contains the `rating_diff` field, which is renamed as Rating Difference. The row shelf contains the `category` field. 

After getting the distribution of first reviews, we want to see how those first review ratings compare to the average rating for a business. The above graph shows the distribution of the rating difference between the user's first review and business's average review in each category. The rating difference would tell us how much higher or lower of a rating a user gave to a business.

The Hotels & Travel category is the only category with a median rating difference far below 0. This cateogry had the lowest median rating difference. This must be because users tend to leave a bad review when experiencing bad quality service during their travels. Food, Nightlife, and Restaurant categories have higher median rating differences. This shows that users tend to want to leave better reviews when they are dining and going out at night.

Looking at this graph, we can see that most of the rating differences have a median of 0 except for the Hotel & Travel category. Our first review would make us expect that these businesses have a strong skew to either the one star or five star sides, but this is not the case. Users have ambivalent feelings when it comes to a business. First reviews make up most of the outliers in the box plots and are typically at the entreme ends of the box plot. This tells us that first reviews are typically biased and will lean toward extremly positive(5 stars) or extremely negative (1 stars).

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/Rating_Difference_by_Metro_Area1.png", 50) )

### Visualization 3: Average Rating Difference

The column shelf contains the `categories` field, where there are 18 categories total. For the row shelf, we had to first create a calculated field. The calculated field looks like: {INCLUDE [Category] : MAX ([Rating_Difference]) }. What this does is for the categories, take the maximum rating difference for each business. Then we aggregated this calculated field by averaging. Finally, the row shelf displays the average rating difference by category.

The scatter plot above shows average rating difference per category in each metro area. The area pictured above, Calgary, is the first of 10 metro areas. On tableau, the pages shelf will appear to scroll through the different areas.

We wanted to see if this is the same pattern geographically for different metro areas. Looking at the 10 metro areas, the average rating difference in most categories will be negative.

## Limitations
* Users with 1 review: excluded those who had a rating other than the 5 rating options (1, 2, 3, 4, 5)
* Users with multiple reviews: we restricted to only using their earliest review in the dataset. This excluded a large portion of the dataset since we will not look at the rest of the reviews by multiple-review users.

## Conclusion

Our team worked towards understanding how the distribution of a user’s first reviews compares to the overall distribution of all reviews for Yelp users. 

By analyzing the first review of users who only wrote one review as well as users who have written multiple reviews we were able to get a more clear understanding of how first reviews impact the distribution of overall reviews. Looking at the first chart we can see the distribution of all the first review data we collected, this chart indicates that most users will create a Yelp account and join to praise a business. The top 3 categories which have the most reviews (Restaurants, Shopping, and Food) have a higher rating than the business' average rating. We can also see that a majority of users that join Yelp usually join to give a business a very low (1 star) or very high (5 stars) rating, there are not a lot of reviews that tend to be in the middle/average range (2-4 stars). 

The data showed us that a large majority of first reviews were written after a longer period rather than immediately after a User created their Yelp account, which also proved to have an impact on the type of reviews that were written. 

Looking at the data we could see that Users that tend to write a review on the same day that they create a Yelp account their first review tends to be more negative, with Users trying to pan a business. Whereas, Users who write their first review after a longer period of time from the day they create an account tend to give businesses a less bias and more positive review. 

In our first review distribution we would expect that businesses have a strong skew to either the one star or five star sides, but this is not the case. Our first reviews are typically biased and will lean toward extremly positive(5 stars) or extremely negative (1 stars). Users have neutral feelings when it comes to a business. First reviews make up most of the outliers in the box plots and are typically at the entreme ends of the box plot. 

Regarding how the review distribution varied by geography, we were able to see that first reviews had a negative rating difference across the top 10 metro areas examined.